# Caching LLM responses

This notebook demonstrates how to use Cassandra for a basic prompt/response cache.

Such a cache prevents running an LLM invocation more than once for the very same prompt, thus saving on latency and token usage. The cache retrieval logic is based on an exact match, as will be shown.

In [1]:
from langchain.cache import CassandraCache

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

Create a `CassandraCache` and configure it globally for LangChain:

In [3]:
import langchain
langchain.llm_cache = CassandraCache(
    session=session,
    keyspace=keyspace,
)

In [4]:
langchain.llm_cache.clear()

Below is the logic to instantiate the LLM of choice. We chose to leave it in the notebooks for clarity.

In [5]:
import os
from llm_choice import suggestLLMProvider

llmProvider = suggestLLMProvider()
# (Alternatively set llmProvider to 'GCP_VertexAI', 'OpenAI', 'Azure_OpenAI' ... manually if you have credentials)

if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    llm = VertexAI()
    print('LLM from Vertex AI')
elif llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from langchain.llms import OpenAI
    llm = OpenAI()
    print('LLM from OpenAI')
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from langchain.llms import AzureOpenAI
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    print('LLM from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM from OpenAI


In [6]:
%%time
SPIDER_QUESTION_FORM_1 = "How many eyes do spiders have?"
# The first time, it is not yet in cache, so it should take longer
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 128 ms, sys: 1.67 ms, total: 130 ms
Wall time: 2.3 s


'\n\nSpiders usually have eight eyes, but some spiders can have as many as twelve eyes.'

In [7]:
%%time
# This time we expect a much shorter answer time
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 2.77 ms, sys: 2.85 ms, total: 5.63 ms
Wall time: 182 ms


'\n\nSpiders usually have eight eyes, but some spiders can have as many as twelve eyes.'

In [8]:
%%time
SPIDER_QUESTION_FORM_2 = "How many eyes do spiders generally have?"
# This will again take 1-2 seconds, being a different string
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 24.8 ms, sys: 5.08 ms, total: 29.9 ms
Wall time: 1.34 s


'\n\nSpiders typically have eight eyes, although some species may have fewer or more.'

### Stale entry control

#### Time-To-Live (TTL)

You can configure a time-to-live property of the cache, with the effect of automatic eviction of cached entries after a certain time.

Setting `langchain.llm_cache` to the following will have the effect that entries vanish in an hour (also supplying a custom table name is demonstrated):

In [9]:
cacheWithTTL = CassandraCache(
    session=session,
    keyspace=keyspace,
    table_name="langchain_llm_cache",
    ttl_seconds=3600,
)

#### Manual cache eviction

Alternatively, you can invalidate cached entries one at a time - for that, you'll need to provide the very LLM this entry is associated to:

In [10]:
%%time
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 7.99 ms, sys: 0 ns, total: 7.99 ms
Wall time: 150 ms


'\n\nSpiders typically have eight eyes, although some species may have fewer or more.'

In [11]:
langchain.llm_cache.delete_through_llm(SPIDER_QUESTION_FORM_2, llm)

In [12]:
%%time
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 13.8 ms, sys: 3.52 ms, total: 17.4 ms
Wall time: 1.02 s


'\n\nMost spiders have eight eyes, although some species have fewer.'

#### Whole-cache deletion

As you might have seen at the beginning of this notebook, you can also clear the cache entirely: **all** stored entries, for all models, will be evicted at once:

In [13]:
langchain.llm_cache.clear()